<a href="https://colab.research.google.com/github/SamuelSousaFerreira/Mercado-Financeiro/blob/main/Case_MZR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Carregando os dados**

In [ ]:
import pandas as pd #operações com tabelas
import numpy as np #operações matemáticas
!pip install xlsxwriter #gerar o excel com várias abas de clientes
import calendar # operações com data
import datetime #operações com data

In [ ]:
arquivo_excel = pd.ExcelFile("/content/drive/MyDrive/Portfólio/Case da MZR/Desafio VBA.xlsx") #Carrego o excel

operações = pd.read_excel(arquivo_excel, sheet_name='Trade Output', header=1)# index_col= 'Date & Time' # coloco em um DataFrame as abas desejadas
desafio = operações.copy()
desafio.head()

,Unnamed: 0,Date & Time,Stock,B/S,Amount,Client,Price
0,NaN,2021-05-07 13:32:31.350,BBDC3,B,5,1040,20.82
1,NaN,2021-05-07 13:32:32.214,PETR4,S,5,1040,25.19
2,NaN,2021-05-07 13:32:33.078,SANB11,S,20,Mesa,38.91
3,NaN,2021-05-07 13:32:33.942,PETR4,S,10,2806,23.54
4,NaN,2021-05-07 13:32:34.806,BBDC3,B,10,2030,21.11


In [ ]:
desafio.drop(columns='Unnamed: 0', inplace=True) #excluindo a coluna em branco
desafio.head() #conferindo

,Date & Time,Stock,B/S,Amount,Client,Price
0,2021-05-07 13:32:31.350,BBDC3,B,5,1040,20.82
1,2021-05-07 13:32:32.214,PETR4,S,5,1040,25.19
2,2021-05-07 13:32:33.078,SANB11,S,20,Mesa,38.91
3,2021-05-07 13:32:33.942,PETR4,S,10,2806,23.54
4,2021-05-07 13:32:34.806,BBDC3,B,10,2030,21.11


In [ ]:
desafio.info()
#Verificando os dados e a tipagem
#não há dados faltantes e nem nulos nas 1000 linhas
#tipagem feita corretamente, portanto não é necessário alterar nada.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date & Time  1000 non-null   datetime64[ns]
 1   Stock        1000 non-null   object        
 2   B/S          1000 non-null   object        
 3   Amount       1000 non-null   int64         
 4   Client       1000 non-null   object        
 5   Price        1000 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 47.0+ KB


In [ ]:
#A coluna B/S, significa B=Buy e S=Sell, então substituiremos B por 1 e S por -1.
compra_venda = {'B':1,'S':-1}
desafio['B/S'] = desafio['B/S'].map(compra_venda)
desafio.head()

,Date & Time,Stock,B/S,Amount,Client,Price
0,2021-05-07 13:32:31.350,BBDC3,1,5,1040,20.82
1,2021-05-07 13:32:32.214,PETR4,-1,5,1040,25.19
2,2021-05-07 13:32:33.078,SANB11,-1,20,Mesa,38.91
3,2021-05-07 13:32:33.942,PETR4,-1,10,2806,23.54
4,2021-05-07 13:32:34.806,BBDC3,1,10,2030,21.11


In [ ]:
#Regra:Posições vendidas devem entrar com sinal negativo nas carteiras.
#Tanto as quantidades quanto o preço receberão o sinal negativo quando a operaçõe for de venda
desafio['Amount'] = desafio['B/S'] * desafio.Amount
desafio['Price'] = desafio['B/S'] * desafio.Price
desafio.head()

,Date & Time,Stock,B/S,Amount,Client,Price
0,2021-05-07 13:32:31.350,BBDC3,1,5,1040,20.82
1,2021-05-07 13:32:32.214,PETR4,-1,-5,1040,-25.19
2,2021-05-07 13:32:33.078,SANB11,-1,-20,Mesa,-38.91
3,2021-05-07 13:32:33.942,PETR4,-1,-10,2806,-23.54
4,2021-05-07 13:32:34.806,BBDC3,1,10,2030,21.11


In [ ]:
#Coluna Volume Negociado
desafio['Volume'] = desafio.Amount * desafio.Price
desafio.head()

,Date & Time,Stock,B/S,Amount,Client,Price,Volume
0,2021-05-07 13:32:31.350,BBDC3,1,5,1040,20.82,104.10
1,2021-05-07 13:32:32.214,PETR4,-1,-5,1040,-25.19,125.95
2,2021-05-07 13:32:33.078,SANB11,-1,-20,Mesa,-38.91,778.20
3,2021-05-07 13:32:33.942,PETR4,-1,-10,2806,-23.54,235.40
4,2021-05-07 13:32:34.806,BBDC3,1,10,2030,21.11,211.10


In [ ]:
# Criando uma lista de Clientes que será nossa base
clientes = list(desafio.Client.unique()) 
clientes

[1040, 'Mesa', 2806, 2030, 1917, 1150, 3018]

# **Mesa**
As boletas registradas sob o operador “Mesa” devem ser especificadas para TODOS os clientes de forma equânime, ou seja, com o mesmo preço médio de forma que não haja seleção de preço por cliente (Fair Dealing).  

In [ ]:
#filtrando as operações da Mesa
mesa = desafio.loc[desafio.Client == 'Mesa']
mesa.head()


,Date & Time,Stock,B/S,Amount,Client,Price,Volume
2,2021-05-07 13:32:33.078,SANB11,-1,-20,Mesa,-38.91,778.20
8,2021-05-07 13:32:38.262,ITUB4,1,15,Mesa,29.43,441.45
41,2021-05-07 13:33:06.774,BBDC3,1,5,Mesa,22.97,114.85
43,2021-05-07 13:33:08.502,BPAC11,1,5,Mesa,112.02,560.10
49,2021-05-07 13:33:13.686,MGLU3,1,15,Mesa,20.80,312.00


## Compra Mesa

In [ ]:
#Quantidade de Ações compradas
mesa_compr = mesa.loc[mesa['B/S'] == 1]
mesa_compr.head()

,Date & Time,Stock,B/S,Amount,Client,Price,Volume
8,2021-05-07 13:32:38.262,ITUB4,1,15,Mesa,29.43,441.45
41,2021-05-07 13:33:06.774,BBDC3,1,5,Mesa,22.97,114.85
43,2021-05-07 13:33:08.502,BPAC11,1,5,Mesa,112.02,560.10
49,2021-05-07 13:33:13.686,MGLU3,1,15,Mesa,20.80,312.00
52,2021-05-07 13:33:16.278,ITUB4,1,10,Mesa,28.68,286.80


In [ ]:
#lista de ações mesa
stock_mesa = list(mesa.Stock.unique())
stock_mesa

['SANB11', 'ITUB4', 'BBDC3', 'BPAC11', 'MGLU3', 'PETR4']

In [ ]:
#Criando um Data Frame para armazenar os Resultados
neg_mesa = pd.DataFrame(columns=['Stock','Amount','Price'])
neg_mesa

,Stock,Amount,Price


In [ ]:
#Calculando a média por ação e armazeando no DF
for x in stock_mesa:
  acoes_x = mesa_compr.loc[mesa_compr.Stock == x]
  volume_x = acoes_x['Volume'].sum()
  negociadas = acoes_x['Amount'].sum()
  preco_medio = (volume_x / negociadas).round(2)
  #print(volume_x, negociadas, preco_medio)
  neg_mesa = neg_mesa.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)


In [ ]:
neg_mesa

,Stock,Amount,Price
0,SANB11,170,39.91
1,ITUB4,150,27.99
2,BBDC3,130,21.34
3,BPAC11,195,111.75
4,MGLU3,205,20.20
5,PETR4,220,24.93
6,SANB11,-185,-39.71
7,ITUB4,-125,-28.36
8,BBDC3,-75,-20.51
9,BPAC11,-190,-111.97


## Resultado da Carteira Mesa

In [ ]:
neg_mesa.groupby('Stock')['Amount'].sum()

Stock
BBDC3     55
BPAC11     5
ITUB4     25
MGLU3     90
PETR4    -25
SANB11   -15
Name: Amount, dtype: int64

## Venda Mesa

In [ ]:
#Ações Vendidas
mesa_vend = mesa.loc[mesa['B/S'] == -1]
mesa_vend.head()

for x in stock_mesa:
  acoes_x = mesa_vend.loc[mesa_vend.Stock == x]
  volume_x = acoes_x['Volume'].sum()
  negociadas = acoes_x['Amount'].sum()
  preco_medio = (volume_x / negociadas).round(2)
  #print(volume_x, negociadas, preco_medio)
  neg_mesa = neg_mesa.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)

In [ ]:
neg_mesa

,Stock,Amount,Price
0,SANB11,170,39.91
1,ITUB4,150,27.99
2,BBDC3,130,21.34
3,BPAC11,195,111.75
4,MGLU3,205,20.20
5,PETR4,220,24.93
6,SANB11,-185,-39.71
7,ITUB4,-125,-28.36
8,BBDC3,-75,-20.51
9,BPAC11,-190,-111.97


## Tudo Junto

In [ ]:
#filtrando as operações da Mesa
mesa = desafio.loc[desafio.Client == 'Mesa']

#lista de ações mesa
stock_mesa = list(mesa.Stock.unique())

#Criando um Data Frame para armazenar os Resultados
neg_mesa = pd.DataFrame(columns=['Stock','Amount','Price'])
neg_mesa

#Ações compradas
mesa_compr = mesa.loc[mesa['B/S'] == 1]
mesa_compr.head()

#Ações Vendidas
mesa_vend = mesa.loc[mesa['B/S'] == -1]
mesa_vend.head()

#Calculando a média por ação e armazeando no DF
for x in stock_mesa:
  acoes_x = mesa_compr.loc[mesa_compr.Stock == x]
  volume_x = acoes_x['Volume'].sum()
  negociadas = acoes_x['Amount'].sum()
  preco_medio = (volume_x / negociadas).round(2)
  #print(volume_x, negociadas, preco_medio)
  neg_mesa = neg_mesa.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)


for x in stock_mesa:
  acoes_x = mesa_vend.loc[mesa_vend.Stock == x]
  volume_x = acoes_x['Volume'].sum()
  negociadas = acoes_x['Amount'].sum()
  preco_medio = (volume_x / negociadas).round(2)
  #print(volume_x, negociadas, preco_medio)
  neg_mesa = neg_mesa.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)

neg_mesa


,Stock,Amount,Price
0,SANB11,170,39.91
1,ITUB4,150,27.99
2,BBDC3,130,21.34
3,BPAC11,195,111.75
4,MGLU3,205,20.20
5,PETR4,220,24.93
6,SANB11,-185,-39.71
7,ITUB4,-125,-28.36
8,BBDC3,-75,-20.51
9,BPAC11,-190,-111.97


# **Para Qualquer Cliente**

In [ ]:
clientes

[1040, 'Mesa', 2806, 2030, 1917, 1150, 3018]

In [ ]:
#Criando uma planilha xlsx para armazenar nas abas as posições de cada cliente
posicoes_clientes = pd.ExcelWriter('posicoes_clientes.xlsx', engine='xlsxwriter') 

for y in clientes:
  a = desafio.loc[desafio.Client == y] #seleciono todas as posições do cliente y
  nome_cliente = 'Client - '+ str(y) 
 
  #lista de ações do cliente
  stock_client = list(a.Stock.unique()) #nome de todas as ações do cliente y

  #Criando um Data Frame vazio para armazenar os Resultados, ele é limpado a cada iteração
  neg_client = pd.DataFrame(columns=['Stock','Amount','Price'])  

  #Ações compradas do cliente y
  a_compr = a.loc[a['B/S'] == 1]  

  #Ações Vendidas do cliente y
  a_vend = a.loc[a['B/S'] == -1]
  
  #Calculando a média ponderada por ação comprada e armazeando no DataFrame
  for x in stock_client:
    acoes_x = a_compr.loc[a_compr.Stock == x]
    volume_x = acoes_x['Volume'].sum()
    negociadas = acoes_x['Amount'].sum()
    preco_medio = (volume_x / negociadas).round(2)
    #print(volume_x, negociadas, preco_medio)
    neg_client = neg_client.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)
     
  #Calculando a média ponderada por ação vendida e armazeando no DataFrame
  for x in stock_client:
    acoes_x = a_vend.loc[a.Stock == x]
    volume_x = acoes_x['Volume'].sum()
    negociadas = acoes_x['Amount'].sum()
    preco_medio = (volume_x / negociadas).round(2)
    #print(volume_x, negociadas, preco_medio)
    neg_client = neg_client.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)
  
  #Regra para eliminar, caso tenha, linhas com posições igual a zero
  neg_client.drop(list(neg_client[neg_client.Amount == 0].index), inplace=True)

  #oraganizando em ordem alfabética
  neg_client.sort_values(by='Stock', ascending=True, inplace=True) 

  # colocando a coluna Stock como índice
  neg_client.set_index('Stock', inplace=True) 

  neg_client.to_excel(posicoes_clientes, sheet_name= nome_cliente) #adicionando as operações em abas do xlsx

  display(nome_cliente, neg_client) #exibindo as operações de cada cliente no período

posicoes_clientes.save()  #salvando o xlsx 
  

'Client - 1040'

,Amount,Price
Stock,,
BBDC3,105,19.87
BBDC3,-220,-20.92
BPAC11,175,112.14
BPAC11,-115,-111.77
ITUB4,290,27.66
ITUB4,-85,-27.92
MGLU3,120,19.44
MGLU3,-160,-19.93
PETR4,285,24.98


'Client - Mesa'

,Amount,Price
Stock,,
BBDC3,130,21.34
BBDC3,-75,-20.51
BPAC11,195,111.75
BPAC11,-190,-111.97
ITUB4,150,27.99
ITUB4,-125,-28.36
MGLU3,205,20.20
MGLU3,-115,-19.41
PETR4,220,24.93


'Client - 2806'

...

,Amount,Price
Stock,,
BBDC3,230,20.41
BBDC3,-210,-20.82
BPAC11,135,111.60
BPAC11,-105,-111.37
ITUB4,170,27.66
ITUB4,-240,-28.44
MGLU3,85,21.17
MGLU3,-230,-20.05
PETR4,315,25.47


'Client - 1150'

,Amount,Price
Stock,,
BBDC3,250,21.29
BBDC3,-215,-21.53
BPAC11,180,112.23
BPAC11,-140,-112.06
ITUB4,195,28.18
ITUB4,-185,-27.89
MGLU3,145,19.82
MGLU3,-120,-20.91
PETR4,235,25.29


'Client - 3018'

,Amount,Price
Stock,,
BBDC3,140,21.53
BBDC3,-175,-20.92
BPAC11,150,112.36
BPAC11,-145,-111.77
ITUB4,195,27.49
ITUB4,-100,-28.11
MGLU3,70,19.61
MGLU3,-150,-19.94
PETR4,155,25.58


In [ ]:
neg_client

,Stock,Amount,Price
0,SANB11,150,40.21
1,BPAC11,150,112.36
2,MGLU3,70,19.61
3,BBDC3,140,21.53
4,ITUB4,195,27.49
5,PETR4,155,25.58
6,SANB11,-130,-39.96
7,BPAC11,-145,-111.77
8,MGLU3,-150,-19.94
9,BBDC3,-175,-20.92


In [ ]:
list(neg_client[neg_client.Amount == 70].index)

[2]

In [ ]:
neg_client.drop(list(neg_client[neg_client.Amount == 70].index), inplace=True)

In [ ]:
neg_client

,Stock,Amount,Price
0,SANB11,150,40.21
1,BPAC11,150,112.36
3,BBDC3,140,21.53
4,ITUB4,195,27.49
5,PETR4,155,25.58
6,SANB11,-130,-39.96
7,BPAC11,-145,-111.77
8,MGLU3,-150,-19.94
9,BBDC3,-175,-20.92
10,ITUB4,-100,-28.11


# **Desafio Adicional**

Desafio adicional:
Uma carteira já possui posições prévias, então a macro deve ser capaz de calcular o novo preço médio considerando as posições já detidas pelo cliente.
Se o cliente por acaso zerar uma posição que tenha na carteira, exclua a linha deste ativo.


In [ ]:
#Carregando o arquivo da carteira prévia
arquivo_excel_2 = pd.ExcelFile("/content/drive/MyDrive/Portfólio/Case da MZR/Desafio VBA.xlsx") #Carrego o excel

Cliente = pd.read_excel(arquivo_excel_2, sheet_name='Client - 1150', header=3)# index_col= 'Date & Time' # coloco em um DataFrame as abas desejadas
cliente_1150 = Cliente.copy()
cliente_1150.drop(columns='Unnamed: 0', inplace= True)
cliente_1150

,Ticker,Amount,Price,Sector
0,BBDC3,20,20.82,Banking
1,PETR4,10,25.19,Oil and Gas
2,SANB11,5,38.91,Banking
3,MGLU3,5,18.48,Retail
4,ITUB4,10,28.07,Banking
5,BPAC11,5,110.50,Banking


In [ ]:
# ações = list(cliente_1150.Ticker.unique())
# ações

In [ ]:
# setor = list(cliente_1150.Sector.unique())
# setor

In [ ]:
#separando os setores para juntá-los no final
# setores = dict(zip(ações,setor))
setores = {"BBDC3":"Banking",'PETR4':'Oil and Gas','SANB11':'Banking','MGLU3':'Retail','ITUB4':'Banking', 'BPAC11':'Banking' }
#setores

{'BBDC3': 'Banking',
 'BPAC11': 'Banking',
 'ITUB4': 'Banking',
 'MGLU3': 'Retail',
 'PETR4': 'Oil and Gas',
 'SANB11': 'Banking'}

In [ ]:
#carregando o arquivo com todas as negociações do cliente. Considerei o arquivo gerado no tópico anterior
arquivo_excel_3 = pd.ExcelFile("/content/posicoes_clientes.xlsx") #Carrego o excel

Cliente = pd.read_excel(arquivo_excel_3, sheet_name='Client - 1150', header=0)# index_col= 'Date & Time' # coloco em um DataFrame as abas desejadas
cliente_1150_2 = Cliente.copy()
cliente_1150_2

,Stock,Amount,Price
0,BBDC3,250,21.29
1,BBDC3,-215,-21.53
2,BPAC11,180,112.23
3,BPAC11,-140,-112.06
4,ITUB4,195,28.18
5,ITUB4,-185,-27.89
6,MGLU3,145,19.82
7,MGLU3,-120,-20.91
8,PETR4,235,25.29
9,PETR4,-205,-25.07


In [ ]:
#renomeando a coluna stock para poder juntar as duas tabelas
cliente_1150_2.rename(columns={'Stock':'Ticker'}, inplace=True)

#juntando as tabelas
cliente_1150_posições = pd.concat([cliente_1150_2,cliente_1150], axis=0)

#excluindo a coluna Setor, mas não é obrigatório
cliente_1150_posições.drop(columns=['Sector'], inplace=True)
                                  
cliente_1150_posições

,Ticker,Amount,Price
0,BBDC3,250,21.29
1,BBDC3,-215,-21.53
2,BPAC11,180,112.23
3,BPAC11,-140,-112.06
4,ITUB4,195,28.18
5,ITUB4,-185,-27.89
6,MGLU3,145,19.82
7,MGLU3,-120,-20.91
8,PETR4,235,25.29
9,PETR4,-205,-25.07


In [ ]:
#criando a coluna volume para o cálculo da média ponderada 
cliente_1150_posições['Volume'] =cliente_1150_posições.Amount * cliente_1150_posições.Price
cliente_1150_posições

,Ticker,Amount,Price,Volume
0,BBDC3,250,21.29,5322.50
1,BBDC3,-215,-21.53,4628.95
2,BPAC11,180,112.23,20201.40
3,BPAC11,-140,-112.06,15688.40
4,ITUB4,195,28.18,5495.10
5,ITUB4,-185,-27.89,5159.65
6,MGLU3,145,19.82,2873.90
7,MGLU3,-120,-20.91,2509.20
8,PETR4,235,25.29,5943.15
9,PETR4,-205,-25.07,5139.35


In [ ]:
#ações presentes na carteira
stock_client = list(cliente_1150_posições.Ticker.unique())
stock_client

['BBDC3', 'BPAC11', 'ITUB4', 'MGLU3', 'PETR4', 'SANB11']

In [ ]:
#ações compradas
a_compr = cliente_1150_posições[cliente_1150_posições.Amount > 0]

In [ ]:
neg_client = pd.DataFrame(columns=['Ticker','Amount','Price'])  
neg_client

,Ticker,Amount,Price


In [ ]:
for x in stock_client:
    acoes_x = a_compr.loc[a_compr.Ticker == x]
    volume_x = acoes_x['Volume'].sum()
    negociadas = acoes_x['Amount'].sum()
    preco_medio = (volume_x / negociadas).round(2)
    #print(volume_x, negociadas, preco_medio)
    neg_client = neg_client.append({"Ticker":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)

In [ ]:
neg_client

,Ticker,Amount,Price
0,BBDC3,270,21.26
1,BPAC11,185,112.18
2,ITUB4,205,28.17
3,MGLU3,150,19.78
4,PETR4,245,25.29
5,SANB11,270,39.96


In [ ]:
#ações vendidas
a_vend = cliente_1150_posições[cliente_1150_posições.Amount < 0]

for x in stock_client:
    acoes_x = a_vend.loc[a_vend.Ticker == x]
    volume_x = acoes_x['Volume'].sum()
    negociadas = acoes_x['Amount'].sum()
    preco_medio = (volume_x / negociadas).round(2)
    #print(volume_x, negociadas, preco_medio)
    neg_client = neg_client.append({"Ticker":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)

In [ ]:
neg_client

,Ticker,Amount,Price
0,BBDC3,270,21.26
1,BPAC11,185,112.18
2,ITUB4,205,28.17
3,MGLU3,150,19.78
4,PETR4,245,25.29
5,SANB11,270,39.96
6,BBDC3,-215,-21.53
7,BPAC11,-140,-112.06
8,ITUB4,-185,-27.89
9,MGLU3,-120,-20.91


In [ ]:
# Aqui considerei colocar o preço médio entre as ações vendidas e compradas para calcular o calor final
#por isso deixei todos os valores positivos
neg_client['Price'] = abs(neg_client.Price)
neg_client

,Ticker,Amount,Price
0,BBDC3,270,21.26
1,BPAC11,185,112.18
2,ITUB4,205,28.17
3,MGLU3,150,19.78
4,PETR4,245,25.29
5,SANB11,270,39.96
6,BBDC3,-215,21.53
7,BPAC11,-140,112.06
8,ITUB4,-185,27.89
9,MGLU3,-120,20.91


In [ ]:
# calculando a posição final e média por ação
neg_client = neg_client.groupby('Ticker').agg({'Amount':sum, 'Price': 'mean'})
neg_client

,Amount,Price
Ticker,,
BBDC3,55,21.395
BPAC11,45,112.120
ITUB4,20,28.030
MGLU3,30,20.345
PETR4,40,25.180
SANB11,190,40.550


In [ ]:
#inserindo a coluna sector
neg_client['Sector'] = neg_client.index.map(setores)
neg_client

,Amount,Price,Sector
Ticker,,,
BBDC3,55,21.395,Banking
BPAC11,45,112.120,Banking
ITUB4,20,28.030,Banking
MGLU3,30,20.345,Retail
PETR4,40,25.180,Oil and Gas
SANB11,190,40.550,Banking


In [ ]:
#Salvando em arquivo excel
#optei por salvar em um diferente mas poderia salvar no mesmo arquivo do tópico antetior 
posicoes_cliente_1150 = pd.ExcelWriter('posicoes_cliente_1150.xlsx', engine='xlsxwriter') 

neg_client.to_excel(posicoes_cliente_1150, sheet_name='Client - 1150')